In [119]:
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()
gezi.set_pandas()

In [120]:
V = 7
root = f'../working/offline/{V}/0'
# pairwise two tower, recall model
# pt_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_ext_emlm_mlm.ep1'
# p2t_model = 'all-mpnet-base-v2.flag-pairwise14-2'
pt_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_mlm3'
# pt_model = 'all-mpnet-base-v2.flag-pairwise14-2'
# pt_model2 = 'pmminilm.flag-pairwise14-2-pre_emlm_mlm-mmnilm'
# pt_model2 = 'pmminilm.flag-pairwise14-2-pre_mlm3-pmminilm'
pt_model2 = 'pmminilm.flag-pairwise14-2'
# pairwise concat, rank model
pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-extpred-ft.neg_rand_prob-0.neg_strategy-rand-sample'
# context model
# c_model = 'deberta-v3-small.flag-context4-2-d'
# c_model2 = 'lsg-mminilm.flag-context4-3-d-s-mminilm'
c_model = 'deberta-v3-small.flag-context4-2-d'

In [121]:
LABEL_COL = 'rel_rank'
df_train = pd.read_feather('../working/train.fea')
df_train = df_train[df_train.fold==0]

In [122]:
xp = gezi.load(f'{root}/{pt_model}.eval/valid.pkl')

In [123]:
xp2 = gezi.load(f'{root}/{pt_model2}.eval/valid.pkl')

In [124]:
x = xp.copy()
rate = 0.9
x['pred'] = rate * xp['cls_pred'] + (1 - rate) * xp2['cls_pred'] 

In [125]:
calc_metric(x)

0.8943555924847786

In [126]:
calc_metric(xp)

0.8906469282447388

In [127]:
calc_metric(xp2)

0.8754903143845476

In [128]:
xp.keys()

dict_keys(['id', 'cell_id', 'cid', 'pred', 'max_prob', 'max_sim', 'cls_pred', 'n_words', 'match_rank', 'match_code', 'n_code_cell', 'n_cell', 'probs', 'sims'])

In [129]:
def gen_feat(x_):
  topn = 10
  x = {}
  keys = ['pred', 'cls_pred']
  x = {k: x_[k] for k in keys}
  x['rank_pred'] = x['pred'] * (1 + x_['n_code_cell']) - 0.5
  x['cls_rank_pred'] = x['cls_pred'] * (1 + x_['n_code_cell']) - 0.5
  x['min_prob'] = x_['probs'].min()
  x['min_sim'] = x_['sims'].min()
  x['var_prob'] = np.var(x_['probs'])
  x['var_sim'] = np.var(x_['sims'])
  idxes = (-x_['probs']).argsort()
  for i in range(topn):
    if i < len(idxes):
      if i > 0:
        x[f'top_pred_{i}'] = (idxes[i] + 0.5) / (x_['n_code_cell'] + 1)
      x[f'top_prob_{i}'] = x_['probs'][idxes[i]]
      x[f'top_sim_{i}'] = x_['sims'][idxes[i]]
    else:
      x[f'top_pred_{i}'] = -1
      x[f'top_prob_{i}'] = -1
      x[f'top_sim_{i}'] = -1
  x['cls_diff'] = x['cls_pred'] - x['pred']
  x['abs_cls_diff'] = abs(x['cls_diff'])
  return x

In [130]:
def gen_feats():
  xs = gezi.batch2list(xp)
  p_feats = [gen_feat(x) for x in tqdm(xs, desc=f'gen_feats:xp')]
  # p_feats = gezi.prun_list(gen_feat, xs, 2, desc=f'gen_feats:xp')
  xs = gezi.batch2list(xp2)
  p2_feats = [gen_feat(x) for x in tqdm(xs, desc=f'gen_feats:xp2')]
  # p2_feats = gezi.prun_list(gen_feat, xs, 2, desc=f'gen_feats:xp2')
  feats = []
  for i in tqdm(range(len(xs)), desc='merge_feats'):
    fe1 = p_feats[i]
    fe2= p2_feats[i]
    
    fe = {}
    fe['code_ratio'] = xp['n_code_cell'][i] / xp['n_cell'][i]
    fe['ps_pred_diff'] = fe1['pred'] - fe2['pred']
    fe['abs_ps_pred_diff'] = abs(fe['ps_pred_diff'])
    fe['ps_cls_pred_diff'] = fe1['cls_pred'] - fe2['cls_pred']
    fe['abs_ps_cls_pred_diff'] = abs(fe['ps_cls_pred_diff'])
    
    fe1 = gezi.dict_prefix(fe1, 'p_')
    fe2 = gezi.dict_prefix(fe2, 'p2_')
    fe.update(fe1)
    fe.update(fe2)
    
    keys = [
     'id', 'cell_id', 'cid',
     'n_words', 'n_code_cell', 'n_cell'
    ]
    for key in keys:
      fe[key] = xp[key][i]
      
    #  ic(feat)
    feats.append(fe)
  #  break
  dfeats = pd.DataFrame(feats)
  return dfeats

In [131]:
dfeats = gen_feats()

gen_feats:xp:   0%|          | 0/424943 [00:00<?, ?it/s]

gen_feats:xp2:   0%|          | 0/424943 [00:00<?, ?it/s]

merge_feats:   0%|          | 0/424943 [00:00<?, ?it/s]

In [132]:
dfeats

,code_ratio,ps_pred_diff,abs_ps_pred_diff,ps_cls_pred_diff,abs_ps_cls_pred_diff,p_pred,p_cls_pred,p_rank_pred,p_cls_rank_pred,p_min_prob,p_min_sim,p_var_prob,p_var_sim,p_top_prob_0,p_top_sim_0,p_top_pred_1,p_top_prob_1,p_top_sim_1,p_top_pred_2,p_top_prob_2,p_top_sim_2,p_top_pred_3,p_top_prob_3,p_top_sim_3,p_top_pred_4,p_top_prob_4,p_top_sim_4,p_top_pred_5,p_top_prob_5,p_top_sim_5,p_top_pred_6,p_top_prob_6,p_top_sim_6,p_top_pred_7,p_top_prob_7,p_top_sim_7,p_top_pred_8,p_top_prob_8,p_top_sim_8,p_top_pred_9,p_top_prob_9,p_top_sim_9,p_cls_diff,p_abs_cls_diff,p2_pred,p2_cls_pred,p2_rank_pred,p2_cls_rank_pred,p2_min_prob,p2_min_sim,p2_var_prob,p2_var_sim,p2_top_prob_0,p2_top_sim_0,p2_top_pred_1,p2_top_prob_1,p2_top_sim_1,p2_top_pred_2,p2_top_prob_2,p2_top_sim_2,p2_top_pred_3,p2_top_prob_3,p2_top_sim_3,p2_top_pred_4,p2_top_prob_4,p2_top_sim_4,p2_top_pred_5,p2_top_prob_5,p2_top_sim_5,p2_top_pred_6,p2_top_prob_6,p2_top_sim_6,p2_top_pred_7,p2_top_prob_7,p2_top_sim_7,p2_top_pred_8,p2_top_prob_8,p2_top_sim_8,p2_top_pred_9,p2_top_prob_9,p2_top_sim_9,p2_cls_diff,p2_abs_cls_diff,id,cell_id,cid,n_words,n_code_cell,n_cell
0,0.8889,0.0000,0.0000,-0.0237,0.0237,0.0556,0.0645,0.0000,0.0808,0.0000,0.5031,0.0867,0.0062,0.9432,0.7505,0.1667,0.0456,0.6840,0.3889,0.0096,0.6498,0.6111,0.0008,0.5963,0.2778,0.0006,0.5899,0.5000,0.0001,0.5559,0.9444,0.0000,0.5262,0.7222,0.0000,0.5204,0.8333,0.0000,0.5031,-1.0000,-1.0000,-1.0000,0.0090,0.0090,0.0556,0.0882,0.0000,0.2937,0.0000,0.2628,0.0591,0.0098,0.7826,0.5662,0.1667,0.1731,0.5248,0.3889,0.0202,0.4660,0.2778,0.0201,0.4658,0.6111,0.0025,0.4087,0.5000,0.0011,0.3866,0.9444,0.0003,0.3546,0.7222,0.0000,0.2721,0.8333,0.0000,0.2628,-1.0000,-1.0000,-1.0000,0.0326,0.0326,0002115f48f982,9ec225f0,0002115f48f982\t9ec225f0,2,8,9
1,0.8333,0.4545,0.4545,0.2323,0.2323,0.5000,0.4358,5.0000,4.2939,0.0004,0.4950,0.0226,0.0026,0.5150,0.6508,0.2273,0.2409,0.6341,0.4091,0.0963,0.6140,0.5909,0.0622,0.6044,0.6818,0.0523,0.6006,0.3182,0.0235,0.5830,0.1364,0.0035,0.5409,0.9545,0.0029,0.5374,0.0455,0.0023,0.5322,0.8636,0.0006,0.5029,-0.0642,0.0642,0.0455,0.2035,0.0000,1.7382,0.0010,0.2431,0.0204,0.0037,0.4471,0.4111,0.1364,0.2928,0.3995,0.5000,0.1636,0.3836,0.6818,0.0494,0.3508,0.9545,0.0199,0.3259,0.2273,0.0179,0.3230,0.3182,0.0041,0.2829,0.7727,0.0017,0.2577,0.4091,0.0015,0.2552,0.5909,0.0011,0.2457,0.1580,0.1580,00062ab8487156,aa354742,00062ab8487156\taa354742,6,10,12
2,0.8333,0.0000,0.0000,0.0088,0.0088,0.9545,0.9456,10.0000,9.9017,0.0000,0.5679,0.0695,0.0052,0.9215,0.8047,0.8636,0.0752,0.7497,0.2273,0.0012,0.6594,0.0455,0.0010,0.6548,0.7727,0.0006,0.6450,0.6818,0.0002,0.6200,0.4091,0.0001,0.5992,0.1364,0.0001,0.5896,0.5909,0.0000,0.5820,0.5000,0.0000,0.5739,-0.0089,0.0089,0.9545,0.9368,10.0000,9.8051,0.0001,0.2140,0.0672,0.0044,0.9087,0.4671,0.8636,0.0623,0.3937,0.6818,0.0144,0.3535,0.7727,0.0043,0.3202,0.0455,0.0036,0.3155,0.2273,0.0026,0.3071,0.5909,0.0019,0.2982,0.1364,0.0011,0.2824,0.5000,0.0007,0.2700,0.3182,0.0004,0.2561,-0.0177,0.0177,00062ab8487156,96c8449c,00062ab8487156\t96c8449c,6,10,12
3,0.5692,0.0000,0.0000,-0.0004,0.0004,0.0132,0.0139,0.0000,0.0264,0.0000,0.3981,0.0249,0.0095,0.9868,0.8596,0.0395,0.0087,0.7556,0.0658,0.0032,0.7340,0.0921,0.0003,0.6848,0.2763,0.0002,0.6746,0.1711,0.0001,0.6648,0.3289,0.0001,0.6519,0.3553,0.0001,0.6504,0.1184,0.0001,0.6480,0.2237,0.0001,0.6425,0.0007,0.0007,0.0132,0.0142,0.0000,0.0403,0.0000,0.0815,0.0253,0.0090,0.9935,0.5934,0.0395,0.0042,0.4435,0.4868,0.0007,0.3938,0.6184,0.0006,0.3890,0.1711,0.0004,0.3766,0.4342,0.0002,0.3534,0.0658,0.0001,0.3504,0.2763,0.0001,0.3423,0.5132,0.0000,0.3201,0.3289,0.0000,0.3191,0.0011,0.0011,000efd285fb982,74a30f80,000efd285fb982\t74a30f80,102,37,65
4,0.5692,0.0000,0.0000,0.0108,0.0108,0.0132,0.0671,0.0000,2.0508,0.0000,0.4218,0.0150,0.0071,0.7658,0.7862,0.0395,0.0883,0.7388,0.4868,0.0491,0.7259,0.0658,0.0250,0.7111,0.5395,0.0133,0.6973,0.6184,0.0120,0.6949,0.0921,0.0113,0.6937,0.5132,0.0105,0.6921,0.1711,0.0101,0.6912,0.1184,0.0032,0.6660,0.0540,0

In [133]:
df = dfeats.merge(df_train[['cid', 'ancestor_id', LABEL_COL]], on='cid')

In [134]:
gezi.set_fold(df, 5, 'ancestor_id')

In [135]:
reg_cols = [x for x in dfeats.columns if x not in ['id', 'cell_id', 'cid', LABEL_COL]]
cat_cols = []
cols = reg_cols + cat_cols
len(cols)

86

In [136]:
fold = 1
dvalid = df[df.fold==fold]
dtrain = df[df.fold!=fold]

In [137]:
X_train = dtrain[cols]
y_train = dtrain[LABEL_COL]
X_valid = dvalid[cols]
y_valid = dvalid[LABEL_COL]

In [138]:
from catboost import CatBoostRegressor, CatBoostRanker
cbt_params = {
              # 'bootstrap_type': 'Poisson',
              # 'learning_rate': 0.02,
              'learning_rate': 0.03,
              'reg_lambda': 7.960622217848342e-07, 
              # 'subsample': 0.7422597612762745,
              # 'bagging_temperature': 0.2,
              'max_depth': 10, 
              'early_stopping_rounds': 100,
              'n_estimators': 10000,
              'cat_features': [],
              'loss_function': 'MAE',
              'min_child_samples': 300,
              # 'task_type': 'GPU',
              # 'devices': '0',
              }

In [139]:
model = CatBoostRegressor(**cbt_params)
model.fit(X_train, y_train,
          eval_set=[(X_valid, y_valid)],
          verbose=100,
          )  
dvalid['cb_pred'] = model.predict(dvalid[cols])
ic(calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values}))

0:	learn: 0.2525273	test: 0.2524315	best: 0.2524315 (0)	total: 73.6ms	remaining: 12m 16s
100:	learn: 0.0768580	test: 0.0764960	best: 0.0764960 (100)	total: 6.6s	remaining: 10m 47s
200:	learn: 0.0707191	test: 0.0704186	best: 0.0704186 (200)	total: 12.9s	remaining: 10m 28s
300:	learn: 0.0695609	test: 0.0695908	best: 0.0695908 (300)	total: 19.2s	remaining: 10m 20s
400:	learn: 0.0687712	test: 0.0692065	best: 0.0692065 (400)	total: 26s	remaining: 10m 21s
500:	learn: 0.0681790	test: 0.0689492	best: 0.0689492 (500)	total: 32.7s	remaining: 10m 20s
600:	learn: 0.0676338	test: 0.0687732	best: 0.0687717 (599)	total: 39.4s	remaining: 10m 16s
700:	learn: 0.0671485	test: 0.0686630	best: 0.0686621 (698)	total: 46s	remaining: 10m 9s
800:	learn: 0.0667656	test: 0.0686057	best: 0.0686057 (800)	total: 53s	remaining: 10m 8s
900:	learn: 0.0664080	test: 0.0685616	best: 0.0685616 (900)	total: 59.6s	remaining: 10m 1s
1000:	learn: 0.0661316	test: 0.0685618	best: 0.0685543 (993)	total: 1m 6s	remaining: 9m 54s
1

[08/04/22 19:39:32] 961508788.py:7 in <module>
                    calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values}): 0.9025058677248895


0.9025058677248895

In [140]:
gezi.plot.feature_importance(model, topn=20)